# IO

## 1. Paths

In [ ]:
const path = require('path');

### 1.1. Normalize

In [ ]:
{
    const dir = 'a/b/c/../d';
    
    const normalizedDir = path.normalize(dir);
    console.log(`* when path is "${dir}", then path.normalize(dir) is: "${normalizedDir}"`);
}

### 1.2. Join

In [ ]:
{
    const dir1 = 'a/b/c';
    const dir2 = '../x/y';
    const fn = 'foo.txt';
    
    const joinedPath = path.join(dir1, dir2, fn);
    console.log(`* when "dir1=${dir1}", "dir2=${dir2}", "fn=${fn}" then "path.join(dir1, dir2, fn)" is: "${joinedPath}"`);
}

### 1.3. Absolute path

In [ ]:
{
    const dir = '../a/b/c';
    
    const absPath = path.resolve(dir);
    console.log(`* when path is "${dir}", then path.resolve(dir) is: "${absPath}"`);
}

### 1.4. Relative path

In [ ]:
{
    const dir1 = 'a/b/c/d/';
    const dir2 = 'a/b/d/';
    
    const re = path.relative(dir1, dir2);
    console.log(`* relative path of "${dir1}" and "${dir2}" is: "${re}"`);
}

### 1.5. Name of directory

In [ ]:
{
    const fullpath = 'a/b/c/d.txt';
    
    const dirname = path.dirname(fullpath);
    console.log(`* dirname of "${fullpath}" is: "${dirname}"`);
}

### 1.6. File name of path

In [ ]:
{
    const fullpath = 'a/b/c/d.txt';
    
    const basename = path.basename(fullpath);
    console.log(`* basename of "${fullpath}" is: "${basename}"`);
}

### 1.7. File extenstion name

In [ ]:
{
    const fullpath = 'a/b/c/d.txt';
    
    const extname = path.extname(fullpath);
    console.log(`* extname of "${fullpath}" is: "${extname}"`);
}

## 2. File operator

In [ ]:
const fs = require('fs');

### 2.1. Remove file

- Function

In [ ]:
async function rm(filename) {
    return await fs.promises.unlink(filename);
}

### 2.2. Is file exist

- Function

In [ ]:
async function exist(filename) {
    try {
        await fs.promises.access(filename);
        return true;
    } catch (err) {
        return false;
    }
}

- Test

In [ ]:
{
    let dir = 'io.ipynb';
    let ok = await exist('io.ipynb');
    console.log(`* "${dir}" file exists? ${ok}`);
    
    dir = path.resolve('../src');
    ok = await exist(dir);
    console.log(`* "${dir}" directory exists? ${ok}`);
}

### 2.3. Create empty file

- Function

In [ ]:
function touch(filename) {
    return new Promise((resolve, reject) => {
        fs.promises.open(filename, 'w')
            .then(h => fs.close(h.fd, () => resolve()))
            .catch(err => reject(err));
    });
}

- Test

In [ ]:
{
    try {
        await touch('test.txt');
        const ok = await exist('test.txt');
        console.log(`* after file created, "test.txt" exist? ${ok}`);
    } finally {
        await rm('test.txt');
    }
}

### 2.4. Create or remove empty folder

- Functions

In [ ]:
async function mkdir(dirname, options = {}) {
    try {
        return await fs.promises.mkdir(dirname, options);
    } catch (err) {
        console.log(err.message);
    }
}

async function rmdir(dirname, options = {}) {
    try {
        return await fs.promises.rmdir(dirname, options);
    } catch (err) {
        console.log(err.message);
    }
}

- Test

In [ ]:
{
    await mkdir('test');
    
    const ok = await exist('test');
    console.log(`* after directory created, "test" exist? ${ok}`);

    await rmdir('test');
}

### 3.5. Remove dirs

- Functions

In [ ]:
const fse = require('fs-extra');

async function mkdirs(pathname) {
    return new Promise((resolve, reject) => {
        fse.mkdirs(pathname, err => {
            if (err) {
                reject(err);
            } else {
                resolve();
            }
        });
    });
}

async function rmdirs(pathname) {
    return new Promise((resolve, reject) => {
        fse.remove(pathname, err => {
            if (err) {
                reject(err);
            } else {
                resolve();
            }
        });
    });
}

- Test

In [ ]:
{
    const dir = 'test/a/b';
    const file = path.join(dir, 'c.txt');
    
    await mkdirs(dir);
    await touch(file);
    
    const ok = await exist(file);
    console.log(`* file "${file}" exist? ${ok}`);
    
    await rmdirs('test');
}

## 3. Buffer

### 3.1. Convert with string

In [ ]:
{
    const strSrc = 'Hello 大家好';
    
    const bufSrc = Buffer.from(strSrc, 'utf-8');
    const data = [...bufSrc];
    console.log(`* convert string "${strSrc}" to bytes are: ${JSON.stringify(data)}`);
    
    const bufDst = Buffer.from(data);
    const strDst = bufDst.toString('utf-8');
    console.log(`* convert data "${JSON.stringify(data)}" to string is: "${strDst}"`);
}

### 3.2. Concat

In [ ]:
{
    const str1 = 'Hello ';
    const str2 = '大家好';
    
    const buf1 = Buffer.from(str1, 'utf-8');
    const buf2 = Buffer.from(str2, 'utf-8');
    
    const bufAll = Buffer.concat([buf1, buf2]);
    const data = [...bufAll];
    
    console.log(`* concat "${JSON.stringify([...buf1])}" and "${JSON.stringify([...buf2])}" is: ${JSON.stringify(data)}`);
    
    const strAll = Buffer.from(data).toString('utf-8');
    console.log(`* convert data "${JSON.stringify(data)}" to string is: "${strAll}"`);
}

### 3.3. Read write data

In [ ]:
const crypto = require('crypto');

{
    let data = null;
    {
        const str = 'hello 大家好';
        const sbuf = Buffer.from(str, 'utf-8');
        const lbuf = Buffer.alloc(4);
        lbuf.writeInt32BE(sbuf.byteLength, 0);
        const cbuf = crypto.createHash('md5').update(sbuf).digest();
        
        data = Buffer.concat([lbuf, sbuf, cbuf]);
        console.log(`* the encoding data is: "${data.toString('hex')}"`);
    }
    
    {
        const len = data.readInt32BE(0);
        console.log(`* the data length is: ${len}`);

        const sbuf = data.slice(4, 4 + len);
        const cbuf = data.slice(4 + len);
        
        const str = sbuf.toString('utf-8');
        console.log(`* the message string in buffer is: "${str}"`);
        
        const checksum = crypto.createHash('md5').update(sbuf).digest();
        console.log(`* calculate checksum is: ${checksum.toString('hex')} and validate checksum is: ${cbuf.toString('hex')}`);
    }
}

### 3.4. Base64

In [ ]:
{
    let base64 = '';
    
    {
        const str = 'hello 大家好';
        const buf = Buffer.from(str, 'utf-8');
        
        base64 = buf.toString('base64');
        console.log(`* string ${str} to base64 is: "${base64}"`);
    }
    
    {
        const buf = Buffer.from(base64, 'base64');
        const str = buf.toString('utf-8');
        console.log(`* base64 "${base64}" to string is: "${str}"`);
    }
}

## 4. File read and write

### 4.1. Read and write file

- Function

In [ ]:
async function fwrite(filename, data) {
    return await fs.promises.writeFile(filename, data);
}

async function fread(filename) {
    return await fs.promises.readFile(filename);
}

async function freadString(filename, encoding='utf-8') {
    return await fs.promises.readFile(filename, encoding);
}

- Test

In [ ]:
{
    const filename = 'test.txt';
    
    try {
        const buf = Buffer.from('Hello 大家好', 'utf-8');
        await fwrite(filename, buf);
        
        const data = await fread(filename);
        console.log(`* read from file: "${data.toString('hex')}" and content is: "${data.toString('utf-8')}"`);
        
        const str = await freadString(filename);
        console.log(`* read from file: "${str}"`);
    } finally {
        await rm(filename);
    }
}

### 4.2. Read and write file from fd

- Functions

In [ ]:
async function lfopen(filename, mode='w+') {
    return await fs.promises.open(filename, mode);
}

async function lfclose(fd) {
    return new Promise((resolve, reject) => {
        fs.close(fd, err => {
            if (err) {
                reject(err);
            } else {
                resolve();
            }
        });
    });
}

async function lfwrite(fd, buf) {
    return new Promise((resolve, reject) => {
         fs.write(fd, buf, (err, written) => {
             if (err) {
                 reject(err);
             } else {
                 resolve(written);
             }
         });
    });
}

async function lfread(fd, buf, offset=0, length=-1, position=0) {
    if (length < 0) {
        length = buf.byteLength;
    }
    return new Promise((resolve, reject) => {
        fs.read(fd, buf, offset, length, position, (err, bytesRead) => {
            if (err) {
                reject(err);
            } else {
                resolve(bytesRead);
            }
        });
    });
}

- Test

In [ ]:
{
    const filename = 'test.txt';
    
    try {
        const wbuf = Buffer.from('Hello 大家好', 'utf-8');

        const wfh = await lfopen(filename);
        const wlen = await lfwrite(wfh.fd, wbuf);
        console.log(`* file written, ${wlen} bytes was written`);
        lfclose(wfh.fd);
        
        const rfh = await lfopen(filename, 'r');
        const rbuf = Buffer.alloc(256);
        const rlen = await lfread(rfh.fd, rbuf);
        console.log(`* file read, ${rlen} bytes was read`);
        lfclose(rfh.fd);
        console.log(`* read content is: ${rbuf.toString('utf-8')}`);
    } finally {
        await rm(filename);
    }
}